##### Creation of a classification model for various paintings to identify the genre, in the dataset as the "TYPE" column, of the painting. <br> I will be using a Convolutional Neural Network with the implementations of ReLU. #####

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [6]:
paint = pd.read_csv("paint_data_labels.csv", sep="\t")
paint.head()

,ID,FILE,AUTHOR,BORN-DIED,TITLE,DATE,TECHNIQUE,LOCATION,FORM,TYPE,SCHOOL,TIMELINE,URL
0,4,4.jpg,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)",Portrait of Emperor Rudolf II,1590s,"Oil on canvas, 60 x 48 cm","Kunsthistorisches Museum, Vienna",painting,portrait,German,1601-1650,http://www.wga.hu/html/a/aachen/rudolf2.html
1,5,5.jpg,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)",Self-Portrait with a Glass of Wine,c. 1596,"Oil on canvas, 53 x 44 cm",Private collection,painting,portrait,German,1601-1650,http://www.wga.hu/html/a/aachen/selfport.html
2,6,6.jpg,"AAGAARD, Carl Frederik","(b. 1833, Odense, d. 1895, København)",Deer beside a Lake,1888,"Oil on canvas, 53 x 82 cm",Private collection,painting,landscape,Danish,1851-1900,http://www.wga.hu/html/a/aagaard/deerlake.html
3,7,7.jpg,"AAGAARD, Carl Frederik","(b. 1833, Odense, d. 1895, København)",The Rose Garden,1877,"Oil on canvas, 98 x 80 cm",Private collection,painting,landscape,Danish,1851-1900,http://www.wga.hu/html/a/aagaard/rosegard.html
4,15,15.jpg,"ABBATE, Niccolò dell'","(b. 1509, Modena, d. 1571, Fontainebleau)",Stag Hunt,1550-52,"Oil on canvas, 116 x 159 cm","Galleria Borghese, Rome",painting,landscape,Italian,1501-1550,http://www.wga.hu/html/a/abbate/deerhunt.html


In [7]:
paint["TYPE"].value_counts()

TYPE
portrait      5640
landscape     4342
still-life    1400
Name: count, dtype: int64

Splitting dataset into training and testing sets

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import shutil

csv_file_path = "paint_data_labels_.csv"  # path to copy of labels 
image_folder = "paint_data_/"  # path to folder of images
testing_image_folder = "paint_test/test_images/"  # path to folder where test images will be stored
training_image_folder = "paint_train/train_images/"  # path to folder where training images will be stored
testing_label_file = "paint_test/test_labels.csv"  # path to test labels
training_label_file = "paint_train/train_labels.csv"  # path to train labels

os.makedirs(testing_image_folder, exist_ok=True)
os.makedirs(training_image_folder, exist_ok=True)

data = pd.read_csv(csv_file_path, sep="\t")
data = data.sort_values(by='ID').reset_index(drop=True)

train_set, test_set = train_test_split(
    data,
    test_size=0.2,
    stratify=data['TYPE'],
    random_state=42
)

train_set = train_set.sort_values(by='ID').reset_index(drop=True)
test_set = test_set.sort_values(by='ID').reset_index(drop=True)

def move_images(data_subset, target_folder):
    for _, row in data_subset.iterrows():
        file_name = row['FILE']
        if pd.notna(file_name):
            src_path = os.path.join(image_folder, file_name)
            dest_path = os.path.join(target_folder, file_name)
            if os.path.exists(src_path):
                shutil.move(src_path, dest_path)
                print(f"Moved file: {file_name} from {src_path} to {dest_path}")
            else:
                print(f"File not found: {file_name} (Skipping)")

print("\nMoving images to training set...")
move_images(train_set, training_image_folder)
print("\nMoving images to testing set...")
move_images(test_set, testing_image_folder)

train_set.to_csv(training_label_file, sep="\t", index=False)
test_set.to_csv(testing_label_file, sep="\t", index=False)

print(f"\nTraining set: {len(train_set)} samples, images moved to {training_image_folder}")
print(f"Testing set: {len(test_set)} samples, images moved to {testing_image_folder}")


Moving images to training set...
Moved file: 4.jpg from paint_data_/4.jpg to paint_train/train_images/4.jpg
Moved file: 5.jpg from paint_data_/5.jpg to paint_train/train_images/5.jpg
Moved file: 6.jpg from paint_data_/6.jpg to paint_train/train_images/6.jpg
Moved file: 7.jpg from paint_data_/7.jpg to paint_train/train_images/7.jpg
Moved file: 15.jpg from paint_data_/15.jpg to paint_train/train_images/15.jpg
Moved file: 19.jpg from paint_data_/19.jpg to paint_train/train_images/19.jpg
Moved file: 21.jpg from paint_data_/21.jpg to paint_train/train_images/21.jpg
Moved file: 29.jpg from paint_data_/29.jpg to paint_train/train_images/29.jpg
Moved file: 30.jpg from paint_data_/30.jpg to paint_train/train_images/30.jpg
Moved file: 36.jpg from paint_data_/36.jpg to paint_train/train_images/36.jpg
Moved file: 50.jpg from paint_data_/50.jpg to paint_train/train_images/50.jpg
Moved file: 62.jpg from paint_data_/62.jpg to paint_train/train_images/62.jpg
Moved file: 64.jpg from paint_data_/64.jpg

Creation of the model

Evaluation